### Tải và import thư viện

In [38]:
%pip install pandas psycopg2 pyodbc sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
import psycopg2
from psycopg2 import sql
import pyodbc
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine
import calendar

### Import Data

In [40]:
csv_file = 'Dataset/bankersData.csv'
df = pd.read_csv(csv_file, encoding='UTF-8')
display(df)

,STT,SoTienDKVayBanDau,TienGiaiNgan,SoTienConLai,application_date,TS_CREDIT_SCORE_V2,Số điện thoại khách hàng,FromDate,ID,LoanID,...,InterestPaymentType,LongestOverdue,CreditInfo,Name,Address,CheckTime,Brieft,NumberOfLoans,HasBadDebt,HasLatePayment
0,2,10000000.0,NaN,0,2023-02-30,403,977966899,2016-06-24 00:00:00,40,16104,...,Lãi ngày,4,NaN,NaN,NaN,2019-03-17 11:22:15,notfound,-1,0,0
1,3,NaN,250000.0,0,2016-07-28,403,977966899,2016-07-28 00:00:00,226,20871,...,Lãi ngày,147,NaN,NaN,NaN,2019-03-17 11:22:15,notfound,-1,0,0
2,4,5000000.0,5000000.0,-50000,2016-06-29,531,397511119,2016-06-29 00:00:00,45,17049,...,Lãi ngày,3,"Khách hàng hiện không có quan hệ tại TCTD, không có nợ cần chú ý và không có nợ xấu tại thời điểm cuối tháng 31/01/2019",DAO CONG QUYET,"P203-E9 PHUONG MAI,DONG DA,HA NOI",2019-03-17 08:29:24,0,0,0,0
3,5,10000000.0,10000000.0,0,2016-06-29,588,abc1234567,2016-06-29 00:00:00,48,17067,...,Lãi ngày,16,"Khách hàng hiện không có quan hệ tại TCTD, không có nợ cần chú ý và không có nợ xấu tại thời điểm cuối tháng 31/01/2019",NGUYEN ANH VU,SO 21 NGACH 77/4 LE HONG PHONG-DIEN BIEN-BA DINH-HN,2019-03-17 08:28:09,0,0,0,0
4,6,10000000.0,10000000.0,0,2016-08-05,675,966710180,2016-08-05 00:00:00,293,22390,...,Lãi ngày,8,"Khách hàng hiện không có quan hệ tại TCTD, không có nợ cần chú ý và không có nợ xấu tại thời điểm cuối tháng 31/01/2019",ĐÀO NGỌC LINH,"-, TT CỤC KHÍ TƯỢNG THỦY VĂN, TRUNG LIỆT, Q.ĐỐNG ĐA, HNO",2019-03-17 14:14:22,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,3181,5000000.0,5000000.0,3474831,2018-12-24,577,353833091,2018-12-24 00:00:00,327685,56967,...,Dư nợ giảm dần cố định ngày đóng lãi,0,"Khách hàng hiện đang quan hệ tại 2 TCTD, không có nợ cần chú ý và không có nợ xấu tại thời điểm cuối tháng 31/01/2019",NGUYỄN HOÀI DUY,"40/8B,,TRẦN XUÂN SOẠN,TÂN THUẬN TÂY,QUẬN 7,TP HỒ CHÍ MINH",2019-03-17 11:57:59,2,2,0,0
2379,3182,3000000.0,3000000.0,3000000,2018-12-22,638,967295514,2018-12-22 00:00:00,329444,56835,...,Tất toán cuối kỳ,2,"Khách hàng hiện đang quan hệ tại 2 TCTD, không có nợ cần chú ý và không có nợ xấu tại thời điểm cuối tháng 31/01/2019",TRẦN HÙNG MẠNH,".,.,TỔ 32,PHƯỜNG NGUYỄN PHÚC,YÊN BÁI,TỈNH YÊN BÁI",2019-03-17 11:52:36,2,2,0,0
2380,3184,6000000.0,6000000.0,0,2018-12-27,605,385669091,2018-12-27 00:00:00,333862,57212,...,Tất toán cuối kỳ,0,"Khách hàng hiện đang quan hệ tại 1 TCTD, không có nợ cần chú ý và không có nợ xấu tại thời điểm cuối tháng 31/01/2019",LUU VAN HANG,"THANH NGHI, THANH XUAN, HA NOI",2019-03-17 11:43:43,1,1,0,0
2381,3187,7000000.0,6000000.0,0,2019-01-10,597,869625266,2019-01-10 00:00:00,348341,57938,...,Dư nợ giảm dần cố định ngày đóng lãi,0,"Khách hàng hiện đang quan hệ tại 2 TCTD, không có nợ cần chú ý và không có nợ xấu tại thời điểm cuối tháng 31/01/2019",NGUYỄN VĂN TRỌNG,THÔN LUẬT NGOẠI 2 - XÃ QUANG LỊCH,2019-03-17 10:47:41,2,2,0,0


In [41]:
for i in range(len(df)):
    # Kiểm tra nếu WardName chỉ chứa số
    if isinstance(df.loc[i, 'DistrictName'], str) and df.loc[i, 'DistrictName'].isdigit():
        df.loc[i, 'DistrictName'] = "Quận " + df.loc[i, 'DistrictName']

    if isinstance(df.loc[i, 'WardName'], str) and df.loc[i, 'WardName'].isdigit():
        df.loc[i, 'WardName'] = "Phường " + df.loc[i, 'WardName']

In [42]:
# Chuyển chuỗi rỗng thành NaN
df['WardName'] = df['WardName'].replace("", None)

# Tạo dictionary từ các giá trị không bị thiếu
district_to_ward = df.dropna().drop_duplicates(subset=["DistrictName", "WardName"]).set_index("DistrictName")["WardName"].to_dict()

ward_to_district = df.dropna().drop_duplicates(subset= ["WardName", 'DistrictName']).set_index("WardName")["DistrictName"].to_dict()

print("Dictionary tự động tạo:")
print(district_to_ward)
print(ward_to_district)

Dictionary tự động tạo:
{'Thanh Trì': 'Vĩnh Quỳnh', 'Nam Từ Liêm': 'Mỹ Đình 1', 'Cầu Giấy': 'Trung Hoà', 'Tây Hồ': 'Phú Thượng', 'Long Biên': 'Thượng Thanh', 'Hoàn Kiếm': 'Chương Dương Độ', 'Bắc Từ Liêm': 'Cổ Nhuế 1', 'Ba Đình': 'Ngọc Khánh', 'Hai Bà Trưng': 'Bạch Mai', 'Đống Đa': 'Trung Phụng', 'Hà Đông': 'Nguyễn Trãi', 'Thanh Xuân': 'Khương Trung', 'Bình Tân': 'Bình Hưng Hòa A', 'Phú Nhuận': 'Phường 3', 'Thủ Đức': 'Linh Xuân', 'Quận 9': 'Tăng Nhơn Phú B', 'Bình Thạnh': 'Phường 22', 'Hoàng Mai': 'Lĩnh Nam', 'Quận 12': 'Tân Chánh Hiệp', 'Gò Vấp': 'Phường 5', 'Quận 3': 'Phường 10'}
{'Tân Triều': 'Thanh Trì', 'Xuân Phương': 'Nam Từ Liêm', 'Yên Hoà': 'Cầu Giấy', 'Tứ Liên': 'Tây Hồ', 'Ngọc Thuỵ': 'Long Biên', 'Phúc Tân': 'Hoàn Kiếm', 'Phú Diễn': 'Bắc Từ Liêm', 'Vĩnh Phúc': 'Ba Đình', 'Thanh Lương': 'Hai Bà Trưng', 'Tả Thanh Oai': 'Thanh Trì', 'Kim Liên': 'Đống Đa', 'Đồng Tâm': 'Hai Bà Trưng', 'Nam Đồng': 'Đống Đa', 'Kiến Hưng': 'Hà Đông', 'Hạ Đình': 'Thanh Xuân', 'Quỳnh Lôi': 'Hai Bà Trưng

In [43]:
df['WardName'] = df['WardName'].fillna(df['DistrictName'].map(district_to_ward))

df['DistrictName'] = df['DistrictName'].fillna(df['WardName'].map(ward_to_district))

In [44]:
df['WardName'] = df['WardName'].fillna(df['DistrictName'])

# Lọc các hàng có giá trị thiếu ở cột 'WardName'
missing_wardname_rows = df[df['DistrictName'].isnull()][['WardName', 'DistrictName']]

# In kết quả
print("Các hàng có giá trị thiếu ở cột 'WardName':")
print(missing_wardname_rows)

Các hàng có giá trị thiếu ở cột 'WardName':
Empty DataFrame
Columns: [WardName, DistrictName]
Index: []


In [45]:
# Chuyển chuỗi rỗng thành NaN
df['CityNameHouseHold'] = df['CityNameHouseHold'].replace("", None)
df['DistrictNameHouseHold'] = df['DistrictNameHouseHold'].replace("", None)
df['WardNameHouseHold'] = df['WardNameHouseHold'].replace("", None)

ward_to_city = df.dropna(subset=['CityNameHouseHold', 'WardNameHouseHold']).drop_duplicates(subset=["WardNameHouseHold", "CityNameHouseHold"]).set_index("WardNameHouseHold")["CityNameHouseHold"].to_dict()

ward_to_district = df.dropna(subset=['DistrictNameHouseHold', 'WardNameHouseHold']).drop_duplicates(subset=["WardNameHouseHold", "DistrictNameHouseHold"]).set_index("WardNameHouseHold")["DistrictNameHouseHold"].to_dict()

ward_to_city.update({
    'Vĩnh Thịnh': 'Vĩnh Phúc',
    'Tân Ninh': 'Quảng Bình',
    'Phú Điền': 'Đồng Nai',
    'Quảng Trạch': 'Quảng Bình',
    'Quảng Cư': 'Sầm Sơn'
})

ward_to_district.update({
    'Vĩnh Thịnh': 'Vĩnh Tường',
    'Tân Ninh': 'Quảng Ninh',
    'Phú Điền': 'Tân Phú',
    'Quảng Trạch': 'Quảng Trạch',
    'Quảng Cư': 'Quảng Cư'

})

print('Dictionary tự động tạo:')
print(ward_to_city)
print(ward_to_district)

Dictionary tự động tạo:
{'Hoàng Liệt': 'Hà Nội', 'Phương Mai': 'Hà Nội', 'Điện Biên': 'Hà Nội', 'Trung Liệt': 'Hà Nội', 'Trung Phụng': 'Hà Nội', 'Phúc La': 'Hà Nội', 'Định Công': 'Hà Nội', 'Trung Hoà': 'Hà Nội', 'Phúc Xá': 'Hà Nội', 'Minh Khai': 'Hà Nội', 'Bồ Đề': 'Hà Nội', 'Phú Lãm': 'Hà Nội', 'Xuân Phương': 'Hà Nội', 'Cổ Nhuế 2': 'Hà Nội', 'Thổ Quan': 'Hà Nội', 'Phú La': 'Hà Nội', 'Gia Thuỵ': 'Hà Nội', 'Dương Nội': 'Hà Nội', 'Thượng Đình': 'Hà Nội', 'Đức Giang': 'Hà Nội', 'Yên Phụ': 'Hà Nội', 'Tương Mai': 'Hà Nội', 'Cổ Nhuế 1': 'Hà Nội', 'Quỳnh Mai': 'Hà Nội', 'Yết Kiêu': 'Hà Nội', 'Hàng Gai': 'Hà Nội', 'Đồng Nhân': 'Hà Nội', 'Khương Mai': 'Hà Nội', 'Khương Đình': 'Hà Nội', 'Quang Trung': 'Hà Nội', 'Quốc Tử Giám': 'Hà Nội', 'Mễ Trì': 'Hà Nội', 'Nhân Chính': 'Hà Nội', 'Trương Định': 'Hà Nội', 'Phạm Đình Hổ': 'Hà Nội', 'Lý Thái Tổ': 'Hà Nội', 'Đống Mác': 'Hà Nội', 'Khương Trung': 'Hà Nội', 'Vĩnh Phúc': 'Hà Nội', 'Tả Thanh Oai': 'Hà Nội', 'Đông Lâm': 'Thái Bình', 'Nghĩa Đô': 'Hà Nội', '

In [46]:
df['CityNameHouseHold'] = df['CityNameHouseHold'].fillna(df['WardNameHouseHold'].map(ward_to_city))

df['DistrictNameHouseHold'] = df['DistrictNameHouseHold'].fillna(df['WardNameHouseHold'].map(ward_to_district))

In [47]:
# Tạo dictionary cho WardNameHouseHold từ CityNameHouseHold và DistrictNameHouseHold
city_district_to_ward = df.dropna(subset=["CityNameHouseHold", "DistrictNameHouseHold", "WardNameHouseHold"]) \
    .drop_duplicates(subset=["CityNameHouseHold", "DistrictNameHouseHold", "WardNameHouseHold"]) \
    .set_index(["CityNameHouseHold", "DistrictNameHouseHold"])["WardNameHouseHold"] \
    .to_dict()

print(district_to_ward)

{'Thanh Trì': 'Vĩnh Quỳnh', 'Nam Từ Liêm': 'Mỹ Đình 1', 'Cầu Giấy': 'Trung Hoà', 'Tây Hồ': 'Phú Thượng', 'Long Biên': 'Thượng Thanh', 'Hoàn Kiếm': 'Chương Dương Độ', 'Bắc Từ Liêm': 'Cổ Nhuế 1', 'Ba Đình': 'Ngọc Khánh', 'Hai Bà Trưng': 'Bạch Mai', 'Đống Đa': 'Trung Phụng', 'Hà Đông': 'Nguyễn Trãi', 'Thanh Xuân': 'Khương Trung', 'Bình Tân': 'Bình Hưng Hòa A', 'Phú Nhuận': 'Phường 3', 'Thủ Đức': 'Linh Xuân', 'Quận 9': 'Tăng Nhơn Phú B', 'Bình Thạnh': 'Phường 22', 'Hoàng Mai': 'Lĩnh Nam', 'Quận 12': 'Tân Chánh Hiệp', 'Gò Vấp': 'Phường 5', 'Quận 3': 'Phường 10'}


In [48]:
df['WardNameHouseHold'] = df['WardNameHouseHold'].fillna(
    df.apply(lambda row: city_district_to_ward.get((row['CityNameHouseHold'], row['DistrictNameHouseHold'])), axis=1)
)

In [49]:
df = df.dropna(subset=['WardNameHouseHold'], how='all')

In [50]:
df['Street'] = df['Street'].fillna('Không có')

In [51]:
df['CityCompany'] = df['CityCompany'].apply(
    lambda x: x if x in ['Hà Nội', 'Hồ Chí Minh'] else np.nan
)

In [52]:
df['CityCompany'] = df['CityCompany'].replace("", None)
df['DistrictNameCompany'] = df['DistrictNameCompany'].replace("", None)

df = df.dropna(subset=['CityCompany', 'DistrictNameCompany'], how='all')

district_to_city = df.dropna(subset=['CityCompany', 'DistrictNameCompany']).drop_duplicates(subset=['DistrictNameCompany', 'CityCompany']).set_index("DistrictNameCompany")["CityCompany"].to_dict()

district_to_city.update({
    'Hải Dương': 'Hải Dương',
    'Vĩnh Yên': 'Vĩnh Yên',
    'Ngô Quyền': 'Hải Phòng'
})

print(district_to_city)

{'Hà Đông': 'Hà Nội', 'Ba Đình': 'Hà Nội', 'Nam Từ Liêm': 'Hà Nội', 'Tây Hồ': 'Hà Nội', 'Hai Bà Trưng': 'Hồ Chí Minh', 'Tiền Hải': 'Hà Nội', 'Thanh Xuân': 'Hà Nội', 'Thanh Trì': 'Hà Nội', 'Gia Lâm': 'Hà Nội', 'Long Biên': 'Hồ Chí Minh', 'Hoàng Mai': 'Hà Nội', 'Đống Đa': 'Hồ Chí Minh', 'Cầu Giấy': 'Hà Nội', 'Văn Chấn': 'Hồ Chí Minh', 'Phú Xuyên': 'Hà Nội', 'Thái Bình': 'Hà Nội', 'Bắc Từ Liêm': 'Hà Nội', 'Hoàn Kiếm': 'Hà Nội', 'Cao Bằng': 'Hà Nội', 'Yên Mỹ': 'Hà Nội', '3': 'Hồ Chí Minh', 'Ba Vì': 'Hà Nội', 'Kbang': 'Hồ Chí Minh', 'Việt Trì': 'Hà Nội', 'Tam Điệp': 'Hà Nội', 'Sông Lô': 'Hà Nội', 'Hoà Bình': 'Hồ Chí Minh', 'Lào Cai': 'Hà Nội', 'An Lão': 'Hà Nội', 'Đô Lương': 'Hà Nội', 'Quỳnh Phụ': 'Hồ Chí Minh', '1': 'Hồ Chí Minh', '5': 'Hồ Chí Minh', '10': 'Hồ Chí Minh', 'Phú Nhuận': 'Hồ Chí Minh', 'Lạc Thủy': 'Hồ Chí Minh', 'Phù Cừ': 'Hà Nội', 'Thái Thụy': 'Hà Nội', 'Cái Bè': 'Hồ Chí Minh', 'Đồng Phù': 'Hồ Chí Minh', '2': 'Hồ Chí Minh', 'Bình Thạnh': 'Hồ Chí Minh', 'Tân Bình': 'Hồ Chí Min

In [53]:
df['CityCompany'] = df['CityCompany'].fillna(df['DistrictNameCompany'].map(district_to_city))

df['DistrictNameCompany'] = df['DistrictNameCompany'].fillna('Not Provided')

#### Phân tích sơ lược các trường trong Dataset

1. Những cột loại bỏ đi do không cần thiết
- Brieft: Mô tả chung
- Name: Tên khách hàng không ảnh hưởng để kết quả phân tích
- Address: Địa chỉ cụ thể đã được thay thế bằng các trường phân cấp khác
- DescriptionPositionJob: Mô tả công việc chi tiết

2. Những cột bị loại bỏ đi do thông tin thiếu hụt quá nhiều và không có phương pháp khôi phục, fillna
- ReceiveYourIncomeSalary: Phương thức chuyển lương

3. Những cột bị loại bỏ đi do trùng lặp thông tin
- SoTienDKVayBanDau: Trùng với cột Số Tiền Đăng Ký Vay Ban Đầu
- TienGiaiNgan: Trùng với cột Tiền Giải Ngân

## Xử lý Moneytary Attribute

In [54]:
df = df.dropna(subset=['Salary'])

In [55]:
df.loc[df['Salary'] == 'Ten thousand', 'Salary'] = '1000$'

df['Salary'] = df['Salary'].apply(lambda x: str(int(x.replace('$', '')) * 25000) if isinstance(x, str) and '$' in x else x)

In [56]:
df.loc[df['Salary'] == '0.0', 'AddressCompany'] = 'Không có'
df.loc[df['Salary'] == '0.0', 'JobName'] = 'Thất nghiệp'
df.loc[df['Salary'] == '0.0', 'NameCompany'] = 'Không có'
df.loc[df['SoTienConLai'] < 0, 'SoTienConLai'] = 0

zero_salary_rows = df[df['Salary'] == '0.0'][['STT', 'JobName', 'NameCompany', 'AddressCompany', 'Salary']]

display(zero_salary_rows)

,STT,JobName,NameCompany,AddressCompany,Salary
70,664,Thất nghiệp,Không có,Không có,0.0
190,644,Thất nghiệp,Không có,Không có,0.0
250,410,Thất nghiệp,Không có,Không có,0.0
255,301,Thất nghiệp,Không có,Không có,0.0
256,302,Thất nghiệp,Không có,Không có,0.0
...,...,...,...,...,...
2032,2630,Thất nghiệp,Không có,Không có,0.0
2046,2647,Thất nghiệp,Không có,Không có,0.0
2121,2741,Thất nghiệp,Không có,Không có,0.0
2252,2930,Thất nghiệp,Không có,Không có,0.0


In [57]:
for ten_cot in ['SoTienConLai',
                'Số tiền đăng ký vay ban đầu', 
                'Tiền giải ngân', 
                'Tiền gốc còn lại', 
                'Salary'
            ]:
    non_numeric = pd.to_numeric(df[ten_cot], errors='coerce')
    non_positive_or_non_numeric = df[non_numeric.isna() | (non_numeric < 0)]
    print(non_positive_or_non_numeric)

Empty DataFrame
Columns: [STT, SoTienDKVayBanDau, TienGiaiNgan, SoTienConLai, application_date, TS_CREDIT_SCORE_V2, Số điện thoại khách hàng, FromDate, ID, LoanID, Số tiền đăng ký vay ban đầu, Tiền giải ngân, Tiền gốc còn lại, ToDate, Trạng thái, FullName, CardNumber, Gender, Birthday, CityName, DistrictName, WardName, Hình thức cư trú, Thời gian đã sống, Street, CityNameHouseHold, DistrictNameHouseHold, WardNameHouseHold, JobName, NameCompany, AddressCompany, CityCompany, DistrictNameCompany, Salary, ReceiveYourIncomeSalary, DescriptionPositionJob, RelativeFamilyName, FullNameFamily, ProductCreditName, InterestPaymentType, LongestOverdue, CreditInfo, Name, Address, CheckTime, Brieft, NumberOfLoans, HasBadDebt, HasLatePayment]
Index: []

[0 rows x 49 columns]
Empty DataFrame
Columns: [STT, SoTienDKVayBanDau, TienGiaiNgan, SoTienConLai, application_date, TS_CREDIT_SCORE_V2, Số điện thoại khách hàng, FromDate, ID, LoanID, Số tiền đăng ký vay ban đầu, Tiền giải ngân, Tiền gốc còn lại, ToD

In [58]:
ten_cot = 'Salary'
df[ten_cot] = pd.to_numeric(df[ten_cot], errors='coerce')

display(df[ten_cot].unique())

non_positive_or_non_numeric = df[~df[ten_cot].apply(pd.to_numeric, errors='coerce').notna() | (df[ten_cot] < 0)]


display(non_positive_or_non_numeric)

array([2.5000000e+07, 7.7000000e+06, 1.2000000e+07, 8.0000000e+06,
       5.0000000e+06, 3.0000000e+07, 1.4000000e+07, 1.5000000e+07,
       6.5000000e+06, 1.0000000e+07, 9.0000000e+06, 7.5000000e+06,
       5.5000000e+06, 3.0000000e+06, 7.6000000e+06, 7.0000000e+06,
       6.0000000e+06, 9.5000000e+06, 3.5000000e+06, 3.7000000e+06,
       4.2000000e+06, 7.3000000e+06, 0.0000000e+00, 1.1000000e+07,
       4.0000000e+06, 1.3000000e+07, 3.9750000e+06, 8.5000000e+06,
       1.2500000e+07, 4.5000000e+06, 1.5200000e+07, 1.7000000e+07,
       9.5600000e+06, 6.3000000e+06, 1.8000000e+07, 9.3000000e+06,
       5.6000000e+06, 2.0000000e+07, 3.8000000e+06, 7.2000000e+06,
       4.9000000e+06, 1.1457000e+07, 1.1400000e+07, 4.1000000e+06,
       2.3500000e+07, 2.3000000e+07, 7.8000000e+06, 9.8000000e+06,
       1.6000000e+07, 7.4000000e+06, 8.2000000e+06, 1.3500000e+07,
       7.4500000e+06, 5.0000000e+07, 6.8000000e+06, 9.9500000e+06,
       4.4000000e+06, 7.1000000e+06, 4.5000000e+07, 1.0000000e

,STT,SoTienDKVayBanDau,TienGiaiNgan,SoTienConLai,application_date,TS_CREDIT_SCORE_V2,Số điện thoại khách hàng,FromDate,ID,LoanID,...,InterestPaymentType,LongestOverdue,CreditInfo,Name,Address,CheckTime,Brieft,NumberOfLoans,HasBadDebt,HasLatePayment


In [59]:
df['Tiền đã trả'] = df['Tiền giải ngân'] - df['Tiền gốc còn lại']

## Scatter NaN Processing

In [60]:
df['ProductCreditName'] = df['ProductCreditName'].fillna('Cầm cố Điện thoại')

In [61]:
df['CheckTime'] = df['CheckTime'].fillna(method='ffill')

C:\Users\admin\AppData\Local\Temp\ipykernel_29976\3626488499.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['CheckTime'] = df['CheckTime'].fillna(method='ffill')


In [62]:
df = df.drop(columns=['Brieft', 'Name', 'Address', 'DescriptionPositionJob', 'ReceiveYourIncomeSalary', 'SoTienDKVayBanDau', 'TienGiaiNgan', 'Hình thức cư trú', 'CreditInfo', 'Street', 'SoTienConLai'])

In [63]:
df['JobName'] = df['JobName'].replace({
    'Tự doanh không ĐKKD': 'Kinh doanh tự do',
    'Kinh doanh tại nhà (không có đăng ký kinh doanh)': 'Kinh doanh tự do',
    'Tự doanh có ĐKKD': 'Kinh doanh tự do',
    'Tự doanh tại chợ': 'Kinh doanh tự do',
    'Các hình thức tự kinh doanh khác': 'Kinh doanh tự do',

    'Kiểm soát': 'Quản lý',
    'Giám Sát': 'Quản lý',

    'Lái xe': 'Vận chuyển',
    'Tài xế': 'Vận chuyển',
    'Shipper(Giao hàng)': 'Vận chuyển',

    'Graber và Khác': 'Khác',

    'Nhân viên tổ chức nhà nước': 'Công chức Nhà nước',
    'CQ Hành chính Nhà nước': 'Công chức Nhà nước',

    'Bảo vệ': 'Công việc dịch vụ',
    'Tạp Vụ/Lao công': 'Công việc dịch vụ',

    'Tài chính': 'Tài chính - Kế toán',
    'Kế toán': 'Tài chính - Kế toán',

    'Công nhân': 'Sản xuất - Kho bãi',
    'Kho' : 'Sản xuất - Kho bãi',

    'Nhân viên bảo hiểm' : 'Tư vấn - Bảo hiểm',
    'Tư vấn' : 'Tư vấn - Bảo hiểm',

    'Nhân sự': 'Hành chính - Nhân sự',
    'Thư ký/Trợ lý': 'Hành chính - Nhân sự',

    'Kỹ thuật (Điện/Nước/Điều hòa)': 'Kỹ sư dân dụng',
    'Kỹ sư (Điện/Nước/Xây dựng/Giao thông)': 'Kỹ sư xây dựng',

    'Thất nghiệp': 'Người tìm việc / công việc không ổn định',
    'Công việc ko ổn định/Thu nhập thất thường': 'Người tìm việc / công việc không ổn định',

    'Giáo viên': 'Giáo dục và Đào tạo',
    'Giảng viên': 'Giáo dục và Đào tạo',
    'Đào tạo': 'Giáo dục và Đào tạo',

    'Bán hàng': 'Bán hàng',
    'Chăm sóc KH': 'Bán hàng',
    'Hỗ trợ Kinh doanh': 'Bán hàng',

    'Kinh doanh': 'Kinh doanh - Marketing',
    'Marketing': 'Kinh doanh - Marketing',

    'Y tá': 'Y tế - Lễ tân',
    'Lễ tân': 'Y tế - Lễ tân'
})

df['Thời gian đã sống'] = df['Thời gian đã sống'].replace('-1', 'Hơn 10 năm')

In [64]:
df['AddressCompany'] = df['AddressCompany'].fillna('Không có')
df['NameCompany'] = df['NameCompany'].fillna('Không có')
df['RelativeFamilyName'] = df['RelativeFamilyName'].fillna('Không có')
df['FullNameFamily'] = df['FullNameFamily'].fillna('Không có')

In [65]:
df.loc[df['NameCompany'].str.contains('Công an|Bộ đội', case=False, na=False), 'JobName'] = 'Nhân viên công an/bộ đội'
df.loc[df['NameCompany'].str.contains('Không có', case=False, na=False) & df['JobName'].isna(), 'JobName'] = 'Không có'
df.loc[df['JobName'].isnull(), 'JobName'] = 'Khác'

df_filtered = df[df['NameCompany'].str.contains('Không có', case=False, na=False) & df['JobName'].isna()][['STT' ,'JobName', 'NameCompany']]

# In ra kết quả
display(df_filtered)

,STT,JobName,NameCompany


In [66]:
df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')
df['application_date'] = df['application_date'].fillna(pd.Timestamp('2023-02-28'))
print(df['application_date'].dtypes)

datetime64[ns]


C:\Users\admin\AppData\Local\Temp\ipykernel_29976\3170493001.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce')


## Xử lý Gender

In [67]:
df['Gender'] = df['Gender'].replace({'Male': 0.0, 'Female': 1.0})
display(df['Gender'].unique())

array(['0.0', 0.0, '1.0'], dtype=object)

## Xử lý Phone Number

In [68]:
df['Số điện thoại khách hàng'] = df['Số điện thoại khách hàng'].apply(lambda x: '0' + str(x) if len(str(x)) == 9 else str(x))

## Xử lý Datetime Attribute

In [69]:
df['Birthday'] = pd.to_datetime(df['Birthday'], errors='coerce')

median_birthday = df['Birthday'].median()

df['Birthday'] = df['Birthday'].fillna(median_birthday)

In [70]:
df['Age'] = df['Birthday'].apply(lambda x: (datetime.now() - x).days // 365)

### Áp dụng MachineLearning: Cột Thời gian đã sống

In [71]:
# Kiểm tra tương quan

numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Ma trận tương quan
correlation_matrix = df[numerical_cols].corr()

# Giá trị duy nhất
unique_values = {col: df[col].nunique() for col in categorical_cols}

correlation_matrix, unique_values

(                                  STT  TS_CREDIT_SCORE_V2        ID    LoanID  \
 STT                          1.000000            0.182521  0.890450  0.745989   
 TS_CREDIT_SCORE_V2           0.182521            1.000000  0.212805  0.170271   
 ID                           0.890450            0.212805  1.000000  0.853149   
 LoanID                       0.745989            0.170271  0.853149  1.000000   
 Số tiền đăng ký vay ban đầu  0.084439            0.037467  0.083060  0.068768   
 Tiền giải ngân               0.076614            0.036696  0.074406  0.060586   
 Tiền gốc còn lại             0.077847            0.033456  0.076672  0.063570   
 CardNumber                   0.073171            0.065491  0.088864  0.086448   
 Salary                       0.129314            0.077046  0.139139  0.081925   
 LongestOverdue              -0.080981           -0.159309 -0.107813 -0.156739   
 NumberOfLoans                0.125822            0.300319  0.153833  0.134475   
 HasBadDebt     

In [72]:
missing_percentage = (df.isnull().mean() * 100).sort_values(ascending=False)

# In kết quả
print("Phần trăm dữ liệu bị thiếu của từng cột:")
print(missing_percentage)

Phần trăm dữ liệu bị thiếu của từng cột:
Thời gian đã sống              6.675116
DistrictNameHouseHold          0.000000
WardNameHouseHold              0.000000
JobName                        0.000000
NameCompany                    0.000000
AddressCompany                 0.000000
CityCompany                    0.000000
DistrictNameCompany            0.000000
Salary                         0.000000
RelativeFamilyName             0.000000
STT                            0.000000
ProductCreditName              0.000000
InterestPaymentType            0.000000
LongestOverdue                 0.000000
CheckTime                      0.000000
NumberOfLoans                  0.000000
HasBadDebt                     0.000000
HasLatePayment                 0.000000
Tiền đã trả                    0.000000
FullNameFamily                 0.000000
CityNameHouseHold              0.000000
application_date               0.000000
Tiền gốc còn lại               0.000000
TS_CREDIT_SCORE_V2             0.000000

In [73]:
pd.set_option('display.max_colwidth', None)

# Lọc các hàng có giá trị thiếu ở cột 'WardName'
missing_rows = df[df['JobName'].isnull()][['STT' ,'JobName', 'NameCompany']]

# In kết quả
print("Các hàng có giá trị thiếu ở cột")
display(missing_rows)

Các hàng có giá trị thiếu ở cột


,STT,JobName,NameCompany


In [74]:
df['STT'] = df['STT'].astype(int)
df['application_date'] = pd.to_datetime(df['application_date'], format='%Y-%m-%d')
df['TS_CREDIT_SCORE_V2'] = df['TS_CREDIT_SCORE_V2'].astype(int)
df['Số điện thoại khách hàng'] = df['Số điện thoại khách hàng'].astype(str)
df['FromDate'] = pd.to_datetime(df['FromDate'], errors='coerce')
df['ID'] = df['ID'].astype(int)
df['LoanID'] = df['LoanID'].astype(int)
df['Số tiền đăng ký vay ban đầu'] = df['Số tiền đăng ký vay ban đầu'].astype(float)
df['Tiền giải ngân'] = df['Tiền giải ngân'].astype(float)
df['Tiền gốc còn lại'] = df['Tiền gốc còn lại'].astype(float)
df['ToDate'] = pd.to_datetime(df['ToDate'], errors='coerce')
df['Trạng thái'] = df['Trạng thái'].astype(str)
df['FullName'] = df['FullName'].astype(str)
df['CardNumber'] = df['CardNumber'].astype(str)
df['Gender'] = df['Gender'].astype(float)
df['Birthday'] = pd.to_datetime(df['Birthday'], format='%Y-%m-%d', errors='coerce')
df['CityName'] = df['CityName'].astype(str)
df['DistrictName'] = df['DistrictName'].astype(str)
df['WardName'] = df['WardName'].astype(str)
df['Thời gian đã sống'] = df['Thời gian đã sống'].astype(str)
df['CityNameHouseHold'] = df['CityNameHouseHold'].astype(str)
df['DistrictNameHouseHold'] = df['DistrictNameHouseHold'].astype(str)
df['WardNameHouseHold'] = df['WardNameHouseHold'].astype(str)
df['JobName'] = df['JobName'].astype(str)
df['NameCompany'] = df['NameCompany'].astype(str)
df['AddressCompany'] = df['AddressCompany'].astype(str)
df['CityCompany'] = df['CityCompany'].astype(str)
df['DistrictNameCompany'] = df['DistrictNameCompany'].astype(str)
df['Salary'] = df['Salary'].astype(float)
df['RelativeFamilyName'] = df['RelativeFamilyName'].astype(str)
df['FullNameFamily'] = df['FullNameFamily'].astype(str)
df['ProductCreditName'] = df['ProductCreditName'].astype(str)
df['InterestPaymentType'] = df['InterestPaymentType'].astype(str)
df['LongestOverdue'] = df['LongestOverdue'].astype(int)
df['CheckTime'] = pd.to_datetime(df['CheckTime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['NumberOfLoans'] = df['NumberOfLoans'].astype(int)
df['HasBadDebt'] = df['HasBadDebt'].astype(bool)
df['HasLatePayment'] = df['HasLatePayment'].astype(bool)
df['Age'] = df['Age'].astype(int)
df['Tiền đã trả'] = df['Tiền đã trả'].astype(float)

In [75]:
datetime_columns = ['application_date', 'FromDate', 'ToDate', 'CheckTime', 'Birthday']

# Vòng lặp xử lý từng cột datetime
for col in datetime_columns:
    df[f'{col}_Year'] = df[col].dt.year
    df[f'{col}_Month'] = df[col].dt.month
    df[f'{col}_Day'] = df[col].dt.day
    df[f'{col}_Weekday'] = df[col].dt.strftime('%A')
    df[f'{col}_Quarter'] = df[col].dt.quarter

    if (df[col].dt.hour != 0).any() or (df[col].dt.minute != 0).any() or (df[col].dt.second != 0).any():
        df[f'{col}_Hour'] = df[col].dt.hour
        df[f'{col}_Minute'] = df[col].dt.minute
        df[f'{col}_Second'] = df[col].dt.second 

## Lưu file 

In [76]:
alter_data = 'Dataset/alter_bankersData.csv'
df.to_csv(alter_data, index=False, encoding='utf-8')

print('Succeed')

Succeed


### Import vào CSDL

In [77]:
conn_str = (
    'mssql+pyodbc://:@MELIST/Bank_Test'
    '?driver=ODBC+Driver+17+for+SQL+Server'
)
engine = create_engine(conn_str)

In [78]:
alter_data = pd.read_csv('Dataset/alter_bankersData.csv')

table_name = 'Bankers'
alter_data.to_sql(table_name, con=engine, if_exists='replace', index=False)

27